In [58]:
#Library Imports
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from keras.preprocessing.image import ImageDataGenerator
import cv2

In [ ]:
#paths...
BASE = "Database"
IMAGES_PATH = os.path.join(BASE, "Image")
LABELS_PATH = os.path.join(BASE, "label.txt")


#labels...
def load_labels(labels_path):
    with open(labels_path, "r") as file:
        lines = file.readlines()
    
    labels = []
    for line in lines:
        parts = line.strip().split()  
        if len(parts) >= 3:  
            try:
                pose = float(parts[1]) 
                depth = float(parts[2])  
                labels.append([pose, depth])  
            except ValueError:
                print(f"Skipping line due to ValueError: {line}")  
        else:
            print(f"Skipping line due to incorrect format: {line}") 
    print(f"Loaded {len(labels)} labels.") 
    return labels


#images...
def load_images(images_path):
    image_paths = []
    images = []
    for filename in os.listdir(images_path):
        if filename.endswith(".jpeg") or filename.endswith(".jpg"):  
            image_paths.append(os.path.join(images_path, filename))
            img = cv2.imread(os.path.join(images_path, filename))
            if img is not None:
                images.append(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))  
    print(f"Loaded {len(images)} images.")  
    return np.array(images), image_paths


#data vis
def plot_label_distribution(labels_df):
    if labels_df.empty:
        print("No data available for plotting.")
        return
    
    plt.figure(figsize=(8, 5))
    plt.hist(labels_df["Pitch"], bins=20, alpha=0.7, label="Pitch")
    plt.hist(labels_df["Roll"], bins=20, alpha=0.7, label="Roll")
    plt.hist(labels_df["Depth"], bins=20, alpha=0.7, label="Depth")
    plt.legend()
    plt.title("Label Distributions")
    plt.show()


def plot_samples(images, labels_df, sample_size=5):
    plt.figure(figsize=(15, 5))
    sample_indices = np.random.choice(len(images), size=sample_size, replace=False)
    for i, idx in enumerate(sample_indices):
        plt.subplot(1, sample_size, i+1)
        plt.imshow(images[idx])
        plt.title(f"Image {labels_df.iloc[idx]['Image']}")
        plt.axis('off')
    plt.show()



# Load data
print("Loading Labels...")
labels = load_labels(LABELS_PATH)

print("Loading Images...")
images, image_paths = load_images(IMAGES_PATH)


if images.size == 0 or not labels:
    raise ValueError("No data loaded. Ensure the dataset paths are correct.")

#parsing labels...
print("Parsing Labels...")
labels = np.array(labels, dtype=np.float32)
y_pose = labels[:, 0]  
depth_labels = labels[:, 1]  

#normalise
X_images = images / 255.0


y_labels_combined = np.stack((y_pose, depth_labels), axis=1)

#splitting data
X_train, X_test, y_train, y_test = train_test_split(X_images, y_labels_combined, test_size=0.2, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=42)


print("Training set shape:", X_train.shape)
print("Validation set shape:", X_val.shape)
print("Test set shape:", X_test.shape)


Loading Labels...
Loaded 2016 labels.
Loading Images...
Loaded 2016 images.
Parsing Labels...


In [62]:
print(f"Images shape: {images.shape}")
print(f"Labels shape: {labels.shape}")


Images shape: (0,)
Labels shape: (0, 3)


In [28]:
#data augmentation...
augmenter = ImageDataGenerator(
    rotation_range=10,
    horizontal_flip=True
)

In [29]:
#CNN classification...

from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization

def build_cnn(input_shape):
    model = Sequential([
        Conv2D(32, (3, 3), activation='relu', input_shape=input_shape),
        MaxPooling2D((2, 2)),
        BatchNormalization(),
        Conv2D(64, (3, 3), activation='relu'),
        MaxPooling2D((2, 2)),
        BatchNormalization(),
        Flatten(),
        Dense(128, activation='relu'),
        Dropout(0.5),
        Dense(2, activation='softmax')
    ])
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    return model

cnn_model = build_cnn((224, 224, 3))



In [ ]:
#training....
from keras.callbacks import ModelCheckpoint

checkpoint = ModelCheckpoint('cnn_best_model.keras', save_best_only=True, monitor='val_accuracy')

history_cnn = cnn_model.fit(
    augmenter.flow(X_train, y_train[:, :2], batch_size=128),
    validation_data=(X_val, y_val[:, :2]),
    epochs=25,
    callbacks=[checkpoint]
)


NameError: name 'X_train' is not defined

In [ ]:
#training visualisation...
def plot_training(history):
    if history is None:
        print("No training history available to plot.")
        return
    
    plt.figure(figsize=(12, 5))
    plt.subplot(1, 2, 1)
    plt.plot(history.history['accuracy'], label='Train Accuracy')
    plt.plot(history.history['val_accuracy'], label='Val Accuracy')
    plt.legend()
    plt.title('Accuracy')

    plt.subplot(1, 2, 2)
    plt.plot(history.history['loss'], label='Train Loss')
    plt.plot(history.history['val_loss'], label='Val Loss')
    plt.legend()
    plt.title('Loss')

    plt.show()


In [32]:
#regression model...
def build_regression_model(input_shape):
    model = Sequential([
        Conv2D(32, (3, 3), activation='relu', input_shape=input_shape),
        MaxPooling2D((2, 2)),
        
        Conv2D(64, (3, 3), activation='relu'),
        MaxPooling2D((2, 2)),
        
        Flatten(),
        Dense(128, activation='relu'),
        Dropout(0.5),
        Dense(1, activation='linear')
    ])
    model.compile(optimizer='adam', loss='mse', metrics=['mae'])
    return model

regression_model = build_regression_model((224, 224, 3))


In [33]:
#training regression...
checkpoint_regression = ModelCheckpoint('regression_best_model.h5', save_best_only=True, monitor='val_loss')

history_regression = regression_model.fit(
    augmenter.flow(X_train, y_train[:, 2], batch_size=128),
    validation_data=(X_val, y_val[:, 2]),
    epochs=25,
    callbacks=[checkpoint_regression]
)


NameError: name 'X_train' is not defined

In [34]:
#evalution + visuallisation of regression...
def plot_regression_results(history):
    plt.figure(figsize=(12, 5))
    plt.subplot(1, 2, 1)
    plt.plot(history.history['mae'], label='Train MAE')
    plt.plot(history.history['val_mae'], label='Val MAE')
    plt.legend()
    plt.title('Mean Absolute Error')

    plt.subplot(1, 2, 2)
    plt.plot(history.history['loss'], label='Train Loss')
    plt.plot(history.history['val_loss'], label='Val Loss')
    plt.legend()
    plt.title('Loss')

    plt.show()

plot_regression_results(history_regression)

NameError: name 'history_regression' is not defined

In [35]:
from sklearn.metrics import classification_report, confusion_matrix

y_pred = cnn_model.predict(X_val)
print(classification_report(y_val[:, :2].argmax(axis=1), y_pred.argmax(axis=1)))
print(confusion_matrix(y_val[:, :2].argmax(axis=1), y_pred.argmax(axis=1)))


NameError: name 'X_val' is not defined

In [36]:
from sklearn.metrics import mean_squared_error

y_pred = regression_model.predict(X_val)
rmse = np.sqrt(mean_squared_error(y_val[:, 2], y_pred))
print(f"RMSE: {rmse:.3f}")


NameError: name 'X_val' is not defined

In [37]:
plt.plot(history_cnn.history['accuracy'], label='Train Accuracy')
plt.plot(history_cnn.history['val_accuracy'], label='Val Accuracy')
plt.legend()
plt.title("CNN Classification Accuracy")
plt.show()


NameError: name 'history_cnn' is not defined